# <center>World Map Geovisualization using Folium and Covid Data

### <center>Instructor: Vinita Silaparasetty

### Import Libraries

In [40]:
import json
import folium
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

### Access Data via API

In [41]:
conn= http.client.HTTPSConnection("api.covid19api.com")
payload=''
headers={}
conn.request("GET","/summary",payload,headers)
res=conn.getresponse()
data=res.read().decode('UTF-8')

### Convert Data to JSON

In [42]:
covid1=json.loads(data)

### Normalize Data 

In [43]:
pd.json_normalize(covid1['Countries'],sep=",");

### Convert to Pandas DataFrame

In [44]:
df=pd.DataFrame(covid1['Countries'])
df;

### Drop Unnecessary Columns

In [45]:
covid2= df.drop(columns=['CountryCode','Slug','Date','Premium'],axis=1)
covid2;

### Generate Base Map

In [46]:
m=folium.Map(tiles="Stamen Terrain", min_zoom=1.5)
m;

### Obtain Geodata

geodata url: https://raw.githubusercontent.com/python-visualization/folium/master/examples/data

geo data file: world-countries.json

In [47]:
url='https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes=f'{url}/world-countries.json'




### Generate Choropleth Map Layer

In [48]:
folium.Choropleth(
    geo_data=country_shapes,
    min_zoom=2,
    name='Covid-19',
    data=covid2,
    columns=['Country','TotalConfirmed'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    nan_fill_color='black',
    legend_name='Total Confirmed Covid Cases',
).add_to(m)
m;

### Generate Circular Markers

In [49]:
covid2.update(covid2['TotalConfirmed'].map('Total Confirmed:{}'.format))
covid2.update(covid2['TotalRecovered'].map('Total Recovered:{}'.format))

coordinates of countries: https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv

In [50]:
coordinates=pd.read_csv('https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv')
coordinates;



In [51]:
covid_final=pd.merge(covid2,coordinates,on='Country')

In [52]:
def plotDot(point):
    folium.CircleMarker(location=[point.latitude,point.longitude],
                        radius=5,
                        weight=2,
                        popup=[point.Country,point.TotalConfirmed,point.TotalRecovered],
                        fil_color='#000000').add_to(m)
                        
                       

In [53]:
covid_final.apply(plotDot,axis=1)
m.fit_bounds(m.get_bounds())
m;

### Generate Base Map

### Generate Heat Map Layer

In [54]:
deaths=covid_final['TotalDeaths'].astype(float)

In [55]:
lat=covid_final['latitude'].astype(float)

In [56]:
lon=covid_final['longitude'].astype(float)

In [57]:
m1.add_child(HeatMap(zip(lat,lon,deaths),radius=0));

### Generate Heat Map Layer

### Generate Circular Marker

In [58]:
def plotDot(point):
    folium.CircleMarker(location=[point.latitude,point.longitude],
                       radius=5,
                       weight=2,
                       popup=[point.Country,point.TotalDeaths],
                       fill_color='#000000').add_to(m1)

In [59]:
covid_final.apply(plotDot,axis=1)
m1.fit_bounds(m1.get_bounds())
m1